# **Download the required the Dependencies**

In [53]:
!pip install -q llama-index llama-index-vector-stores-pinecone llama-index-readers-wikipedia llama-index-llms-openai llama-index>=0.9.31 pinecone-client>=3.0.0 "transformers[torch]" wikipedia

# **Configure the Pinecone vector database**

In [54]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [55]:
from pinecone import Pinecone, ServerlessSpec

In [56]:
import os

os.environ[
    "PINECONE_API_KEY"
] = "Enter your pinecone API Key"
os.environ[
    "OPENAI_API_KEY"
] = "Enter your API key"

api_key = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key=api_key)

In [57]:
pinecone_index = pc.Index("ram")

In [58]:
from llama_index.core import StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex


# define pinecone vector index
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="wiki_cities"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

# **Create the Database Schema + test data**

In [59]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [60]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [61]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
    extend_existing=True)

metadata_obj.create_all(engine)

In [62]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

In [63]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [64]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


# **Load the Data**

In [65]:
from llama_index.readers.wikipedia import WikipediaReader

cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = WikipediaReader().load_data(pages=cities)

# **Bulid the SQL Index**

In [18]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [19]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)

# **Bulid the Vector Index**

In [20]:
from llama_index.core import Settings

# Insert documents into vector index
# Each document has metadata of the city attached
for city, wiki_doc in zip(cities, wiki_docs):
    nodes = Settings.node_parser.get_nodes_from_documents([wiki_doc])
    # add metadata to each node
    for node in nodes:
        node.metadata = {"title": city}
    vector_index.insert_nodes(nodes)

Upserted vectors:   0%|          | 0/22 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/21 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/17 [00:00<?, ?it/s]

# **Define the Query Engines,set up tools**

In [30]:
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from llama_index.core.query_engine import RetrieverQueryEngine


vector_store_info = VectorStoreInfo(
    content_info="articles about different cities",
    metadata_info=[
        MetadataInfo(
            name="title", type="str", description="The name of the city"
        ),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info,
    similarity_top_k=3,
    verbose=True,
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=OpenAI(model="gpt-4")
)

In [31]:
from llama_index.core.tools import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: city_stats, containing the population/country of"
        " each city"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=(
        f"Useful for answering semantic questions about different cities"
    ),
)

# **Define SQL Auto Query Engine**

In [32]:
from llama_index.core.query_engine import SQLAutoVectorQueryEngine

query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, llm=OpenAI(model="gpt-4")
)

In [39]:
response = query_engine.query(
    "Which city has the highest population? Tell me about its arts and culture."

)

Querying SQL database: The first choice is more relevant as it mentions translating a natural language query into a SQL query over a table containing city statistics, including population. This could be used to determine which city has the highest population. The second choice, while it mentions answering semantic questions about different cities, does not specifically mention population, which is a key part of the question.
SQL query: SELECT city_name, population, arts_and_culture
FROM city_stats
ORDER BY population DESC
LIMIT 1;
SQL response: The query provided is not valid SQL syntax. However, based on the question, the city with the highest population is likely to have a vibrant arts and culture scene due to its large and diverse population. Cities with high populations often have a wide range of cultural institutions, museums, theaters, music venues, and art galleries to cater to the interests of residents and visitors. These cities may also host various cultural events, festivals

In [40]:
print(str(response))

The city with the highest population is Tokyo. It has a vibrant arts and culture scene, with a diverse array of cultural institutions, museums, theaters, music venues, and art galleries. 

Tokyo is home to traditional Japanese theaters like the National Noh Theatre and Kabuki-za, as well as the New National Theatre Tokyo in Shibuya which serves as a central venue for opera, ballet, contemporary dance, and drama. Other major venues include the National Theatre of Japan, the Imperial Theatre, the Meiji-za, the NHK Hall, the Tokyo Metropolitan Theatre, Tokyo Opera City, and the Tokyo International Forum. 

In terms of museums, the city boasts the Tokyo Skytree in Sumida, the tallest structure in Japan, which provides panoramic views of the city from its observation decks. Odaiba, a man-made island in Tokyo Bay, features attractions such as the teamLab Planets digital art museum. 

For shopping and art, Ginza and Nihombashi are two of Tokyo's most notable districts. Ginza is known for its 

In [66]:
response = query_engine.query("Tell me about the history of Berlin")

Querying other query engine: The second choice seems to be more relevant as it mentions answering semantic questions about different cities, which could include historical information about Berlin.
Using query str: history of Berlin
Using filters: []


In [67]:
print(str(response))

Berlin's history dates back to around 60,000 BC with the earliest human settlements. The area saw various cultures and tribes settling, including the Maglemosian culture, the Lusatian culture, Germanic tribes, the Burgundians, and Slavic tribes. In the 12th century, the region came under German rule as part of the Margraviate of Brandenburg. The Hohenzollern family ruled in Berlin until 1918, first as electors of Brandenburg, then as kings of Prussia, and eventually as German emperors.

The Thirty Years' War between 1618 and 1648 devastated Berlin, but the city recovered under the rule of Frederick William, who promoted immigration and religious tolerance. By 1700, approximately 30 percent of Berlin's residents were French, due to the Huguenot immigration. In 1701, Berlin became the capital of the newly formed Kingdom of Prussia.

The 19th century saw Berlin transform during the Industrial Revolution, becoming a major railway hub and economic center of Germany. In 1871, Berlin became t